### Imports and path

In [ ]:
# lib
import sys

sys.path.append("..")

# set cwd one up
import os

os.chdir("..")
path = "data/hes/Hes1_example.csv"

os.getcwd()

In [ ]:
from gpcell import utils, plots
import numpy as np

### Setting up params

In [ ]:
par1 = np.array([300, 1, 0.07, 0.07, 1, 1, 0], dtype=np.float64)
par2 = np.array([100, 3, 0.03, 0.03, 1, 1, 18], dtype=np.float64)
n_cells = 500  # total replicates used in the simulation
t_final_vals = [1500, 600]  # 25/10 hours
noise_vals = [np.sqrt(x) for x in [0.1, 0.5]]

sim_params = [[noise, t_final_vals[0], n_cells] for noise in noise_vals]
sim_params.append([noise_vals[0], t_final_vals[1], n_cells])

path = "data/simulation/"
paths = [
    path + f"noise_{round(noise**2, 2)}_time_{t_final}_rep_{n_cells}.csv"
    for noise, t_final, n_cells in sim_params
]


### Simulation

In [ ]:
# --- generate data for plot --- #
# XY = [
#     utils.get_time_series(par1, par2, t_final, noise, n_cells, path=path, mode="x")
#     for path, (noise, t_final, n_cells) in zip(paths, sim_params)
# ]

for path, (noise, t_final, n_cells) in zip(paths, sim_params):
    utils.get_time_series(par1, par2, t_final, noise, n_cells, path=path, mode="x")

# # save data
# for xy, path in zip(XY, paths):
#     utils.save_sim(xy[0], xy[1], path)


In [ ]:
# from gpcell.backend import _simulate_replicate_mod9

# --------------------------------#
# --- test simulate functions --- #
# --------------------------------#

# --- test speed --- #
# %timeit XY = [utils.get_time_series(par1, par2, t_final, noise, n_cells) for noise, t_final, n_cells in sim_params]

# --- test load data --- #
# import pandas as pd
# df = pd.read_csv(paths[0])
# print(df.drop(columns="Time").shape[1])

# --- test simulate --- #
# X, Y = utils.get_time_series(par1, par2, t_final_vals[0], noise_vals[0], n_cells)
# print(len(X), len(Y))
# print(X[0].shape, Y[0].shape)

### ROC Dev

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
from gpcell import OscillatorDetector

# --- Function parameters --- #
filename = paths[0]
noise, t_final, n_cells = sim_params[0]
joblib = True


# ----------------------------#
# --- Function definition --- #
# ----------------------------#

# --- Read the generated data  --- #
# The CSV file is assumed to have a "Time" column (in hours) and then 2*n_cells columns for the simulated data.
x, y_list = utils.get_time_series(
    par1, par2, t_final, noise, n_cells, path=filename, mode="x"
)
total_columns = len(y_list)

if total_columns != 2 * n_cells:
    raise ValueError(
        f"Expected 2*n_cells = {2 * n_cells} columns, but CSV has {total_columns} columns."
    )


In [ ]:
from gpcell.backend.priors import sim_ou_prior, sim_ouosc_prior

# This detector will perform the model fitting and return BIC differences.
params = {
    "verbose": True,
    "joblib": joblib,
    "plots": ["BIC"],
    "set_noise": noise,
    "detrend": False,
    "ou_prior_gen": sim_ou_prior,
    "ouosc_prior_gen": sim_ouosc_prior,
}
od = OscillatorDetector.from_file(filename, "Time", "Cell", "Cell", params=params)
od.fit("BIC")
BICdiffM = od.BIC_diffs  # list or array with length equal to total_columns
BICdiffTOT = np.array(BICdiffM)

In [ ]:
od.bic_plot

In [ ]:
# --- ROC analysis using BIC differences --- #

# Split BIC differences into two groups: A (first n_cells) and B (next n_cells)
A = BICdiffTOT[:n_cells]
B = BICdiffTOT[n_cells : 2 * n_cells]
print(A.shape, B.shape)

# Define a threshold vector spanning a little below the min to a little above the max.
thresh = np.linspace(BICdiffTOT.min() - 1, BICdiffTOT.max() + 1, 200)
FP1 = np.zeros_like(thresh)
TP1 = np.zeros_like(thresh)
for i, th in enumerate(thresh):
    FP1[i] = np.sum(A > th)
    TP1[i] = np.sum(B > th)

FP1

In [ ]:
# --- ROC analysis using Lomb–Scargle beat detection ---#
d_noise, d_t, d_n = sim_params[0]
print(d_noise, d_t / 60, d_n * 2)

# Define thrvec as the exponential of 200 points linearly spaced between -15 and -0.01
thrvec = np.exp(np.linspace(-15, -0.01, 200))

# Preallocate arrays for FP2 and TP2. Their length is the same as thrvec.
FP2 = np.zeros(len(thrvec))
TP2 = np.zeros(len(thrvec))

# Preallocate beatvec for the number of columns in dataTOT.
# (dataTOT is assumed to have shape (n_samples, 2*CellNum))
beatvec = np.zeros(total_columns)

# Loop over threshold values
for i, thr in enumerate(thrvec):
    # For each signal in dataTOT:
    for j, y in enumerate(y_list):
        # preprocess and compute the Lomb-Scargle periodogram
        y_processed = y  # (y - np.mean(y)) / np.std(y)
        ls = LombScargle(x, y, normalization="standard")

        # Compute power at frequencies, then compute false alarm probability at them
        freq, power = ls.autopower()
        p_vals = ls.false_alarm_probability(power)

        # If any p-value is below the threshold, set the corresponding beatvec element to 1
        beatvec[j] = int(np.any(p_vals < thr))

    # Split detections into two groups, analogous to MATLAB’s A and B.
    A = beatvec[:n_cells]
    B = beatvec[n_cells : 2 * n_cells]
    FP2[i] = np.sum(A)
    TP2[i] = np.sum(B)


In [ ]:
# --- (Optional) Plot ROC curves ---
plt.figure()
plt.plot(FP1 / n_cells, TP1 / n_cells, label="GP")
plt.plot(FP2 / n_cells, TP2 / n_cells, label="L-S")
plt.xlabel("1 - Specificity (false positive rate)")
plt.ylabel("Sensitivity (true positive rate)")
plt.title("ROC Curve")
plt.legend(loc="center right")
plt.show()


### ROC and plotting

In [ ]:
# calculate ROC
fp1_vals, fp2_vals, tp1_vals, tp2_vals = [], [], [], []
for path, (noise, t_final, n_cells) in zip(paths, sim_params):
    fp1, tp1, fp2, tp2 = plots.compute_rocs_from_file(path, noise, n_cells, joblib=True)
    fp1_vals.append(fp1)
    tp1_vals.append(tp1)
    fp2_vals.append(fp2)
    tp2_vals.append(tp2)

In [ ]:
import pandas as pd

# plot ROC
x_arr = []
dataNormed_arr = []

for path in paths:
    df = pd.read_csv(path)

    x = df["Time"].values
    dataNormed = df.drop(columns=["Time"]).values

    x_arr.append(x)
    dataNormed_arr.append(dataNormed)

plots.plot_rocs_and_timeseries(
    x_arr[0],
    dataNormed_arr[0],
    fp1_vals[0],
    tp1_vals[0],
    fp2_vals[0],
    tp2_vals[0],
    x_arr[1],
    dataNormed_arr[1],
    fp1_vals[1],
    tp1_vals[1],
    fp2_vals[1],
    tp2_vals[1],
    x_arr[2],
    dataNormed_arr[2],
    fp1_vals[2],
    tp1_vals[2],
    fp2_vals[2],
    tp2_vals[2],
    n_cells,
)

In [ ]:
x, y_list = utils.get_time_series(
    par1, par2, t_final, noise, n_cells, path=filename, mode="r"
)
total_columns = len(y_list)

if total_columns != 2 * n_cells:
    raise ValueError(
        f"Expected 2*n_cells = {2 * n_cells} columns, but CSV has {total_columns} columns."
    )


# Given parameters
thrvec = np.exp(np.linspace(-15, -0.01, 200))
n_cells = len(y_list) // 2

# Preallocate arrays for FP and TP
FP2 = []
TP2 = []

# Convert false detection probability to Astropy false alarm probability
for i, thr in enumerate(thrvec):
    false_alarm_prob = thr
    beatvec = np.zeros(len(y_list))

    if i % 10 == 0:
        print(f"Processing threshold {i} of {len(thrvec)}")

    for j, y1 in enumerate(y_list):
        # Handle possible NaNs in data
        mask = ~np.isnan(y1)
        x_clean = x[mask]
        y1_clean = y1[mask]

        ls = LombScargle(x_clean, y1_clean, normalization="standard")
        frequency, power = ls.autopower(minimum_frequency=0.1, maximum_frequency=10)
        threshold = ls.false_alarm_level(false_alarm_prob)
        beatvec[j] = np.any(power > threshold)

    # Compute false positives and true positives
    A = beatvec[:n_cells]
    B = beatvec[n_cells : 2 * n_cells]
    FP2.append(np.sum(A))
    TP2.append(np.sum(B))

# Plot the ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(FP2, TP2, "-o", markersize=3)
plt.xlabel("False Positives")
plt.ylabel("True Positives")
plt.title("ROC Curve (Lomb-Scargle Classification)")
plt.grid(True)
plt.show()


In [ ]:
# Define thrvec as the exponential of 200 points linearly spaced between -15 and -0.01
thrvec = np.exp(np.linspace(-15, -0.01, 200))

# Preallocate arrays for FP2 and TP2. Their length is the same as thrvec.
FP2 = np.zeros(len(thrvec))
TP2 = np.zeros(len(thrvec))

# Preallocate beatvec for the number of columns in y_list
beatvec = np.zeros(total_columns)

# Loop over threshold values
for i, thr in enumerate(thrvec):
    # For each signal in dataTOT:
    for j, y in enumerate(y_list):
        ls = LombScargle(x, y, normalization="standard")
        freq, power = ls.autopower()
        p_vals = ls.false_alarm_probability(power)
        beatvec[j] = int(np.any(p_vals < thr))

    # Split detections into two groups, analogous to MATLAB’s A and B.
    A = beatvec[:n_cells]
    B = beatvec[n_cells : 2 * n_cells]
    FP2[i] = np.sum(A)
    TP2[i] = np.sum(B)


# --- (Optional) Plot ROC curves ---
plt.figure()
plt.margins(x=0, y=0)
plt.plot(FP2 / n_cells, TP2 / n_cells, label="L-S")
plt.xlabel("1 - Specificity (false positive rate)")
plt.ylabel("Sensitivity (true positive rate)")
plt.title("ROC Curve")
plt.legend(loc="center right")
plt.show()